In [ ]:
language = 'pt'

from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

# ===============================
# GRAVAÇÃO DE ÁUDIO (JS)
# ===============================
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  display(Javascript(RECORD))
  js_result = output.eval_js('record(%s)' % (sec * 1000))
  audio = b64decode(js_result.split(',')[1])

  arquivo_voz = 'request_audio.wav'
  with open(arquivo_voz, 'wb') as abrir:
    abrir.write(audio)

  return f'/content/{arquivo_voz}'


# ===============================
# OUVINDO O USUÁRIO
# ===============================
print('🎤 Ouvindo...\n')
record_file = record()
display(Audio(record_file, autoplay=False))


# ===============================
# WHISPER - TRANSCRIÇÃO
# ===============================
!pip install git+https://github.com/openai/whisper.git -q
import whisper

model = whisper.load_model("small")
result = model.transcribe(record_file, fp16=False, language=language)
transcription = result["text"]

print("📝 Transcrição:", transcription)


# ===============================
# OPENAI CONFIG
# ===============================
!pip install openai
import os
import openai
from getpass import getpass

# Solicita a API Key sem expor no código
os.environ["OPENAI_API_KEY"] = getpass("Digite sua OpenAI API Key: ")
openai.api_key = os.environ["OPENAI_API_KEY"]


# =====================================================
#  PROMPT DE SISTEMA + MEMÓRIA DE CONVERSA
# =====================================================
conversation_history = [
    {
        "role": "system",
        "content": (
            "Você é uma assistente de voz educada, clara e objetiva. "
            "Explique conceitos de forma simples e direta, como para iniciantes."
        )
    }
]


# =====================================================
# COMANDOS DE VOZ
# =====================================================
transcription_lower = transcription.lower()

if "limpar conversa" in transcription_lower:
    conversation_history.clear()
    conversation_history.append({
        "role": "system",
        "content": "Você é uma assistente de voz educada e objetiva."
    })
    chatgpt_response = "Conversa limpa com sucesso. Podemos começar de novo."

elif "encerrar assistente" in transcription_lower:
    chatgpt_response = "Assistente encerrado. Até mais!"

else:
    # Adiciona a fala do usuário ao histórico
    conversation_history.append({
        "role": "user",
        "content": transcription
    })

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=conversation_history
    )

    chatgpt_response = response.choices[0].message.content

    # Adiciona resposta ao histórico
    conversation_history.append({
        "role": "assistant",
        "content": chatgpt_response
    })

print("🤖 Assistente:", chatgpt_response)


# ===============================
# TEXTO → VOZ (gTTS)
# ===============================
!pip install gTTS
from gtts import gTTS

gtts_object = gTTS(
    text=chatgpt_response,
    lang=language,
    slow=False
)

response_audio = "/content/response_audio.wav"
gtts_object.save(response_audio)

display(Audio(response_audio, autoplay=True))
